解析大模型vllm推理的结果

In [2]:
import re
import json
import datasets
from datasets import load_dataset

In [6]:
import sys
sys.path.append('..')

from settings import NAME2IDX

In [3]:
infer_dataset = load_dataset(
    "json",
    data_files="output/vllm_ag_news_test.json",
    split="train",
)

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
infer_dataset

Dataset({
    features: ['prompt', 'predict', 'label'],
    num_rows: 1000
})

In [7]:
# Custom parser
def extract_json(text: str):
    pattern = r"```json(.*?)```"
    matches = re.search(pattern, text, re.DOTALL)
    if matches:
        res = matches.group(1)
        try:
            d = json.loads(res.strip())
        except Exception as e:
            print(e.args)
            d = {}
        finally:
            return d
    else:
        return {}

In [8]:
infer_dataset[0]

{'prompt': 'system\nYou are a helpful assistant.\nuser\n\n    Classify the given text into exactly one category, providing clear reasoning for your choice. The available categories are: World, Sports, Business, Science/Technology.\n    Text to classify:\n    Indian board plans own telecast of Australia series The Indian cricket board said on Wednesday it was making arrangements on its own to broadcast next month #39;s test series against Australia, which is under threat because of a raging TV rights dispute.\n    Output the classification reasoning in reason and the selected category in label. Return the response in JSON format as follows:\n    ```json\n    {\n        "reason" : "Classification rationale",\n        "label" : "Text classification label (select exactly one): ["World", "Sports", "Business", "Science/Technology"]"\n    }\n    ```\n    \nassistant\n',
 'predict': '```json\n{\n    "reason" : "The text discusses the Indian cricket board\'s plans to broadcast a test series aga

In [9]:
extract_json(infer_dataset[0]["predict"])

{'reason': "The text discusses the Indian cricket board's plans to broadcast a test series against Australia, which is a sporting event. It also mentions a TV rights dispute, which is relevant to sports broadcasting. Therefore, this text falls under the Sports category.",
 'label': 'Sports'}

In [21]:
def trans_dataset(item):
    try:
        llm_pred_obj = extract_json(item["predict"])
    except:
        llm_pred_obj = {"reason": "<error>", "label": "<error>"}

    item["gold_label"] = item["label"].strip()
    item["label"] = NAME2IDX.get(item["label"].strip())

    item["pred_label"] = llm_pred_obj["label"]
    item["reason"] = llm_pred_obj["reason"]
    return item
infer_dataset

Dataset({
    features: ['prompt', 'predict', 'label'],
    num_rows: 1000
})

In [23]:
output_dataset = infer_dataset.map(trans_dataset)
output_dataset

Dataset({
    features: ['prompt', 'predict', 'label', 'gold_label', 'pred_label', 'reason'],
    num_rows: 1000
})

In [24]:
output_dataset[0]

{'prompt': 'system\nYou are a helpful assistant.\nuser\n\n    Classify the given text into exactly one category, providing clear reasoning for your choice. The available categories are: World, Sports, Business, Science/Technology.\n    Text to classify:\n    Indian board plans own telecast of Australia series The Indian cricket board said on Wednesday it was making arrangements on its own to broadcast next month #39;s test series against Australia, which is under threat because of a raging TV rights dispute.\n    Output the classification reasoning in reason and the selected category in label. Return the response in JSON format as follows:\n    ```json\n    {\n        "reason" : "Classification rationale",\n        "label" : "Text classification label (select exactly one): ["World", "Sports", "Business", "Science/Technology"]"\n    }\n    ```\n    \nassistant\n',
 'predict': '```json\n{\n    "reason" : "The text discusses the Indian cricket board\'s plans to broadcast a test series aga

In [25]:
output_dataset.to_json("../output/vllm_infer.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1551203